In [1]:
### process the final_raw_noauto modeling output into formats that could be easily read into R
import torch
import pickle
import numpy as np
import pandas as pd

In [2]:
with open('tomtom_final_raw_noauto_data4viz.pkl','rb') as f:
    [hmeans,smeans,dmeans,stor_grp,stor_grp_prb,stor_dim_prb,grp_map,dim_map] = pickle.load(f)


In [3]:
# # massage from 9,15,4 array of 15,4 tensors to 9,15,4,15,4,
# def torch2np(mat):
#     newsize = list(mat.shape)
#     newsize.extend(list(mat[0,0,0].shape))
#     newmat = np.empty(shape = newsize)
#     enu = np.ndenumerate(mat)
#     for i in enu:
#         ind = i[0]
#         val = i[1]
#         newmat[ind] = val.detach().numpy()
#     return newmat
# hmeans = torch2np(hmeans)
# smeans = torch2np(smeans)
# dmeans = torch2np(dmeans)

# def dicttensor2dictarray(dic):
#     for i in dic.keys():
#         dic[i] = dic[i].detach().numpy()
#     return dic
# grp_map = dicttensor2dictarray(grp_map)
# dim_map = dicttensor2dictarray(dim_map)

#### extract components

extract each component from both group and dimension models, save to separate csvs

In [4]:
[dim_comp_1, dim_comp_2, dim_comp_3] = [
    torch.distributions.Beta(dim_map['topic_a'][i],dim_map['topic_b'][i]).mean.detach() for i in range(len(dim_map['topic_a']))
]

[grp_comp_1, grp_comp_2, grp_comp_3] = [
    torch.distributions.Beta(grp_map['alpha'][i],grp_map['beta'][i]).mean.detach() for i in range(len(grp_map['alpha']))
]

#### classification/dimension weights to csv

save input-classification/weight associations to csv

In [5]:
enu = np.ndenumerate(stor_grp)
full = []
for ele in enu:
    i,j,k = ele[0]
    grp_h = ele[1]
    grp_s = stor_grp_prb[i,j,k].detach().numpy()
    dim = stor_dim_prb[i,j,k].detach().numpy()
    row = [i,j,k,grp_h]
    row.extend(grp_s)
    row.extend(dim)
    full.append(row)

colnames = ['step','from','to','grp_h']
[colnames.append('grp_s_{}'.format(i + 1)) for i in range(3)]
[colnames.append('dim_{}'.format(i + 1)) for i in range(3)]
df = pd.DataFrame(full, columns = colnames)

In [6]:
# load in states
states = pd.read_csv('C:/Users/zhaoz/group-inference/data/states.csv') # there's probably a better place to put this
states['statepair'] = states['state1'] + '_' + states['state2']# construct state pair strings
# remove autotransiitons
states_noauto = states.loc[states.state1 != states.state2].reset_index(drop = True)
# repeat 9 times to match up with the full dataframe
nrep = df.shape[0]//states_noauto.shape[0]
expanded_states = states_noauto.copy()
for i in range(nrep-1):
    expanded_states = expanded_states.append(states_noauto)

# first merge with the main frame
df = pd.concat([df.reset_index(drop = True),expanded_states.reset_index(drop = True)], axis = 1)

In [7]:
# merge with original states to form final data frame. auto transitions all columns coded -1
temps = []
for i in df.step.unique():
    temp = df[df.step == i]
    temp_wstate = states.merge(
        temp,how = 'left',on = ['state1','state2','set','statepair']
    )
    # fill the NAs in step
    temp_wstate['step'] = i
    temps.append(temp_wstate)
    
df = pd.concat(temps,axis = 0).reset_index(drop = True)

In [8]:
# save to file
df.to_csv('infer_unobserved.csv', index = False)

In [9]:
# save components in long format after adding state information
def process_component(comp):
    a = pd.DataFrame(comp.detach().numpy()).melt(ignore_index = False,var_name = 'to')
    a['from'] = a.index
    a = a.sort_values(['from','to'])
    a = pd.concat([a.reset_index(drop = True),states_noauto.reset_index(drop = True)], axis = 1)
    b = states.merge(
        a,how = 'left',on = ['state1','state2','set','statepair']
    )
    return b

In [10]:
dim_comp_1 = process_component(dim_comp_1)
dim_comp_2 = process_component(dim_comp_2)
dim_comp_3 = process_component(dim_comp_3)
grp_comp_1 = process_component(grp_comp_1)
grp_comp_2 = process_component(grp_comp_2)
grp_comp_3 = process_component(grp_comp_3)

dim_comp_1.to_csv('dim_comp_1.csv', index = False) 
dim_comp_2.to_csv('dim_comp_2.csv', index = False) 
dim_comp_3.to_csv('dim_comp_3.csv', index = False) 
grp_comp_1.to_csv('grp_comp_1.csv', index = False) 
grp_comp_2.to_csv('grp_comp_2.csv', index = False) 
grp_comp_3.to_csv('grp_comp_3.csv', index = False) 